## Run and monitor training experiment on Azure (and optionally publish trained model)

### Imports

In [1]:
# %matplotlib inline
# import matplotlib.pyplot as plt

import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.18


### Parameters

In [2]:
# Ml service / model to be trained
ml_service =   'TWV'  #'PAD' #"TWV"
model_name =   'SS01' #'A001'#'T001' #'SS01' #'M010'#'M005' #"T001" #"SS01"
model_version =  '011' # '001' # '002' #'010' #'002' #"001" #"009" # None for latest - although currently None doesn't seem to work on Azure
clip_training_set = -1 # Dont clip
clip_test_set = -1 # Dont clip

#Hyperparameters for neural network
learning_rate =  0.001
iters = 2000
hidden_layers = [100,100]
regularization = 1.0

#Hyperparameters for isolation forest
iso_num_estimators = 200
iso_max_samples = 'auto'
iso_max_features = '1.0'


### Settings

In [3]:
#Experiment
experiment_name = "train_" + ml_service + '_' + model_name + '_' + model_version
print('Experiment name: ',experiment_name)

#Script locations
relative_script_folder = 'azure_upload_scripts'
training_script_file_name = 'train_model.py'
ngamlfpy_package_name = 'ngamlfpy'

#other
required_conda_packages = ['scikit-learn','pandas','matplotlib','numpy']
data_folder = './data'
workspace_config_file = 'azure_config_dev.json'

Experiment name:  train_TWV_SS01_011


### Create workspace, experiment, datastore objects

In [4]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config(path=workspace_config_file)
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

exp = Experiment(workspace=ws, name=experiment_name)
print(exp.name)

ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)
print(ds.path(data_folder).as_mount())

Found the config file in: C:\Users\russellM\OneDrive - Northgate Information Solutions Limited\Documents\GitLab\AzureML\nga_mlpy\azure_config_dev.json
twv_test-ws	australiaeast	POCML	australiaeast
train_TWV_SS01_011
AzureBlob twvtestws9297627118 azureml-blobstore-bb811392-f07a-47a4-8d37-dd1a5a5aa90f
$AZUREML_DATAREFERENCE_7b308a6f2fa34c12842062a665d853fc


### Get (or create)  compute target

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpucluster") #cpucluster #"aml-compute" # "try-gpu"
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 1) #1 to get ready machine
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2") #"STANDARD_NC6")#"STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. cpucluster


### Prepare scripts

In [6]:
#script folder
import os
script_folder = os.path.join(os.getcwd(), relative_script_folder)
os.makedirs(script_folder, exist_ok=True)
import shutil


from distutils.dir_util import copy_tree
files_in_ngamlfpy_copied = copy_tree(ngamlfpy_package_name, os.path.join(script_folder,ngamlfpy_package_name))

print('Files in package ',ngamlfpy_package_name,' copied: ')
for file_copied in files_in_ngamlfpy_copied:
    print('  ',file_copied)

print(' ')    
print('Script copied: ')    
shutil.copy(training_script_file_name,script_folder)

Files in package  ngamlfpy  copied: 
   C:\Users\russellM\OneDrive - Northgate Information Solutions Limited\Documents\GitLab\AzureML\nga_mlpy\azure_upload_scripts\ngamlfpy\algorithms.py
   C:\Users\russellM\OneDrive - Northgate Information Solutions Limited\Documents\GitLab\AzureML\nga_mlpy\azure_upload_scripts\ngamlfpy\hrxmlconfig.py
   C:\Users\russellM\OneDrive - Northgate Information Solutions Limited\Documents\GitLab\AzureML\nga_mlpy\azure_upload_scripts\ngamlfpy\outputs\PAD_model_A001_001.pkl
   C:\Users\russellM\OneDrive - Northgate Information Solutions Limited\Documents\GitLab\AzureML\nga_mlpy\azure_upload_scripts\ngamlfpy\pipeline.py
   C:\Users\russellM\OneDrive - Northgate Information Solutions Limited\Documents\GitLab\AzureML\nga_mlpy\azure_upload_scripts\ngamlfpy\train.py
   C:\Users\russellM\OneDrive - Northgate Information Solutions Limited\Documents\GitLab\AzureML\nga_mlpy\azure_upload_scripts\ngamlfpy\utils.py
   C:\Users\russellM\OneDrive - Northgate Information Sol

'C:\\Users\\russellM\\OneDrive - Northgate Information Solutions Limited\\Documents\\GitLab\\AzureML\\nga_mlpy\\azure_upload_scripts\\train_model.py'

### Prepare training estimator

In [7]:
model  = model_name

from azureml.train.estimator import Estimator

script_params = {
    '--data-folder': ds.path(data_folder).as_mount(),
    '--model'             : model,
    '--ml_service'        : ml_service,
    '--model-version'     : model_version,
    '--clip_training_set' : clip_training_set,
    '--clip_test_set'     : clip_test_set,
    
    #only relevant for neural network based ml services:
    '--learning-rate' : learning_rate,
    '--iters'         : iters,
    '--hidden-layers' : hidden_layers,
    '--regularization': regularization,
   #only relevant for isolation forest based ml services: 
    '--iso-num-estimators': iso_num_estimators,
    '--iso-max-samples': iso_max_samples,
    '--iso-max-features': iso_max_features

 
    
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script=training_script_file_name,
                conda_packages=required_conda_packages)

### Submit training run experiment

In [8]:
run = exp.submit(config=est)
run

Experiment,Id,Type,Status,Details Page,Docs Page
train_TWV_SS01_011,train_TWV_SS01_011_1585884574_dd621b24,azureml.scriptrun,Starting,Link to Azure Portal,Link to Documentation


### Monitor training run

In [9]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [10]:
# specify show_output to True for a verbose log
run.wait_for_completion(show_output=False) 

{'runId': 'train_TWV_SS01_011_1585884574_dd621b24',
 'target': 'cpucluster',
 'status': 'Finalizing',
 'startTimeUtc': '2020-04-03T03:30:13.473227Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'dcefdd29-374a-48dc-8ac4-1c1b87375727',
  'AzureML.DerivedImageName': 'azureml/azureml_47c164f4b09750356d7cb5558152fe9b',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'runDefinition': {'script': 'train_model.py',
  'useAbsolutePath': False,
  'arguments': ['--data-folder',
   '$AZUREML_DATAREFERENCE_4a4a3ceda23643f88b19a6cbe8881371',
   '--model',
   'SS01',
   '--ml_service',
   'TWV',
   '--model-version',
   '011',
   '--clip_training_set',
   '-1',
   '--clip_test_set',
   '-1',
   '--learning-rate',
   '0.001',
   '--iters',
   '2000',
   '--hidden-layers',
   '100',
   '100',
   '--regularization',
   '1.0',
   '--iso-num-estimators',
   '200',
   '--iso-max-samples',
   'auto',
  

## Register Model if training run is sufficiently successful

### Get metrics and prepare tags

In [11]:
print(run.get_metrics())

{'ML Service': 'TWV', 'Model Name': 'SS01', 'Training start time': '2020-04-03 03:30:46.144930', 'regularization rate': 1.0, 'Acc train': 24.85256950294861, 'Acc test': 24.410774410774412, 'cost train': 4593.326608256108, 'cost test': 4024.1551505050493, 'iters': 2000, 'batch size': 200, 'learning rate': 0.001, 'num hidden layers': 2, 'iso num estimators': 200, 'iso max samples': 'auto', 'iso max features': 1.0, 'num train examples': 2374, 'algorithm': 'scikit_neural_network_regressor', 'training_code_version': '0.1d', 'Training end time': '2020-04-03 03:31:42.995088', 'Cost per iteration': None, 'Train set - Percentage diff per employee': None, 'Train set  abs perc differences (actual vs predicted Tax) per employee': None, 'Train set - Percentage diff per employee Histogram': None, 'Test set - Percentage diff per employee': None, 'Test set - Percentage differences': None, 'Train set - Percentage differences': None}


In [12]:
run_num = 5  # Need to manually set run num, as isn't shown in metrics

In [13]:
from ngamlfpy.hrxmlconfig import MLModelConfig
metrics = run.get_metrics()
dets = run.get_details()

#TODO - replace with call to HRXML Config API to get algorithm details for web service
if ml_service == 'TWV':
    algorithm = 'DNN_MLPRegressor'
    description = 'Tax Withholding Verification'
elif ml_service == 'PAD':
    algorithm = 'Isolation_Forest'
    description = 'Payroll Anomaly detection'
else:
    algorithm = 'Other algorithm'
    description = 'Other'
    
tags = {'RunNumber':run_num,'RunId':dets['runId'],'Model':metrics['Model Name'],'ModelVersion':model_version,'TrainSetSize':metrics['num train examples'],'Algorithm':algorithm,'AlgorithmType':metrics['algorithm']}

if algorithm == 'Isolation_Forest':
    tags['ISONumEstimators'] = metrics['iso num estimators']
    tags['ISOMaxSamples'] = metrics['iso max samples']
    tags['ISOMaxFeatures'] = metrics['iso max features']
else:
    tags['HiddenLayers'] = metrics['num hidden layers']
    tags['LearningRate'] = metrics['learning rate']
    tags['Iters'] = metrics['iters']         

model_config = MLModelConfig.get_model_config_from_web_service_for_model(ml_service,model)

if model_config:
    cat_feats = len(model_config.get_feature_field_names_with_type('C'))
    num_feats = len(model_config.get_feature_field_names_with_type('N'))
    tags['NumFeatures'] = cat_feats + num_feats
else:
    tags['NumFeatures'] = 0
tags

web service call successful:  api/customer-models 
web service call successful:  api/customer-models/6 
web service call successful:  api/model-info ?ml_service=TWV&system=PP9&gcc=SOL&lcc=SUS&variant=BIWK


{'RunNumber': 5,
 'RunId': 'train_TWV_SS01_011_1585884574_dd621b24',
 'Model': 'SS01',
 'ModelVersion': '011',
 'TrainSetSize': 2374,
 'Algorithm': 'DNN_MLPRegressor',
 'AlgorithmType': 'scikit_neural_network_regressor',
 'HiddenLayers': 2,
 'LearningRate': 0.001,
 'Iters': 2000,
 'NumFeatures': 20}

### Register model

In [14]:
# register model 

azure_model_name = ml_service + '_model_' + model + '_' + model_version

reg_model = run.register_model(model_name=azure_model_name, model_path='outputs/' + ml_service + '_model_' + model + '_' + model_version + '.pkl',tags=tags) #, description = description)
print(reg_model.name, reg_model.id, reg_model.version, sep='\t')
 

TWV_model_SS01_011	TWV_model_SS01_011:1	1
